# 🚀 VTrackAI Studio - GPU Backend (SAM3)

This notebook runs the VTrackAI backend on Colab GPU with real SAM3 video segmentation.

**Requirements:**
- Colab GPU runtime (T4/A100)
- ngrok account (free tier works)

**Steps:**
1. Run all cells
2. Copy the ngrok URL
3. Set `VITE_API_URL=<url>` in your local frontend

## 1. Check GPU

In [ ]:
import torch
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU - Go to Runtime > Change runtime type > GPU")

## 2. Install Dependencies

In [ ]:
!pip install -q fastapi uvicorn[standard] python-multipart
!pip install -q opencv-python librosa huggingface_hub
!pip install -q pyngrok nest-asyncio

## 3. Install SAM3

In [ ]:
!git clone https://github.com/facebookresearch/sam3.git
!pip install -q -e ./sam3

## 4. Clone VTrackAI

In [ ]:
!git clone https://github.com/Amitro123/vtrack-ai-studio.git
%cd vtrack-ai-studio/backend

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 5. Download SAM3 Checkpoint

In [ ]:
from huggingface_hub import hf_hub_download
import os

os.makedirs("checkpoints/sam3", exist_ok=True)
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3-hiera-large",
    filename="sam3_hiera_large.pt",
    local_dir="./checkpoints/sam3"
)
print(f"✅ Checkpoint: {checkpoint_path}")

## 6. Set up ngrok

In [ ]:
NGROK_AUTHTOKEN = "YOUR_NGROK_TOKEN_HERE"  # @param {type:"string"}

from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTHTOKEN)

## 7. Launch Backend 🚀

In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

nest_asyncio.apply()
public_url = ngrok.connect(8000).public_url

print("\n" + "="*50)
print(f"🚀 VTrackAI GPU Backend LIVE!")
print(f"📡 URL: {public_url}")
print("="*50)
print(f"\n👉 Frontend: VITE_API_URL={public_url}")

uvicorn.run("server:app", host="0.0.0.0", port=8000)